In [1]:
import sys
package_path = 'EfficientNet-PyTorch/'
sys.path.append(package_path)

from efficientnet_pytorch import EfficientNet
from radam import RAdam, PlainRAdam, AdamW
from am_softmax import AMSoftmaxLoss, AngleSimpleLinear

In [2]:
import numpy as np 
import pandas as pd
from PIL import Image
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.utils.data as D
from torch.optim.lr_scheduler import ExponentialLR
from torchvision import models, transforms as T
import torch.nn.functional as F

from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers.tqdm_logger import ProgressBar
from ignite.handlers import  EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [3]:
!nvidia-smi

Sun Aug 18 03:45:27 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            On   | 00000000:00:04.0 Off |                    0 |
| N/A   79C    P0    29W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
config = {
    'SEED': 42,
    'CLASSES': 1108,
    'PATH_DATA': '/home/tienen/kaggle_dataset_drugs/',
    'DEVICE': 'cuda',
    'BATCH_SIZE': 8,
    'VAL_SIZE': 0.05,
    'MODEL_NAME': 'EfficientNet_b2_AMSLoss_RAdam',
    'USE_BN': True,
    'USE_ANGULAR': True,
    'LR': 1e-3,
    'LR_STR': '1e-3',
    'TURN_OFF_ON_N_EPOCHS': 0,
}

best_epoch = 11

In [4]:
def seed_torch(seed=42):
    import random; import os
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(config['SEED'])

In [6]:
class ImagesDS(D.Dataset):
    def __init__(self, df, img_dir, mode='train', site=1, channels=[1,2,3,4,5,6]):
        self.records = df.to_records(index=False)
        self.channels = channels
        self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]
        
    @staticmethod
    def _load_img_as_tensor(file_name):
        with Image.open(file_name) as img:
            return T.ToTensor()(img)

    def _get_img_path(self, index, channel):
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir,self.mode,experiment,f'Plate{plate}',f'{well}_s{self.site}_w{channel}.png'])
        
    def __getitem__(self, index):
        paths = [self._get_img_path(index, ch) for ch in self.channels]
        img = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths])
        if self.mode == 'train':
            return img, int(self.records[index].sirna)
        else:
            return img, self.records[index].id_code

    def __len__(self):
        return self.len

In [7]:
df = pd.read_csv(path_data+'/train.csv')
df_train, df_val = train_test_split(df, test_size=config['VAL_SIZE'],
                                    stratify=df.sirna, random_state=config['SEED'])
df_test = pd.read_csv(path_data+'/test.csv')

In [9]:
ds_1 = ImagesDS(df_train, path_data, site=1, mode='train')
ds_2 = ImagesDS(df_train, path_data, site=2, mode='train')
ds = D.ConcatDataset([ds_1, ds_2])

ds_val_1 = ImagesDS(df_val, path_data, site=1, mode='train')
ds_val_2 = ImagesDS(df_val, path_data, site=2, mode='train')
ds_val = D.ConcatDataset([ds_val_1, ds_val_2])

ds_test_1 = ImagesDS(df_test, path_data, site=1, mode='test')
ds_test_2 = ImagesDS(df_test, path_data, site=2, mode='test')

In [10]:
train_loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = D.DataLoader(ds_val, batch_size=batch_size, shuffle=True, num_workers=4)

test_loader_1 = D.DataLoader(ds_test_1, batch_size=1, shuffle=False, num_workers=4)
test_loader_2 = D.DataLoader(ds_test_2, batch_size=1, shuffle=False, num_workers=4)

In [11]:
class EffNet(nn.Module):
    def __init__(self, num_classes=1000, num_channels=6, use_bn=False, use_angular=False):
        super().__init__()
        self.use_angular = use_angular
        self.use_bn = use_bn
        if self.use_bn:
            self.bn = nn.BatchNorm2d(6)
        
        self.features = EfficientNet.from_pretrained('efficientnet-b2', num_classes=num_classes)
        # print(self.features)
        
        trained_kernel = self.features._conv_stem.weight
        new_conv = nn.Sequential(nn.Conv2d(num_channels, 32, kernel_size=(3,3), stride=(2,2), bias=False),
                    nn.ZeroPad2d(padding=(0, 1, 0, 1)))
        with torch.no_grad():
            new_conv[0].weight[:,:] = torch.stack([torch.mean(trained_kernel, 1)]*6, dim=1)
        self.features._conv_stem = new_conv
        
        if self.use_angular:
            self.features._fc = AngleSimpleLinear(1408, num_classes)
        
    def forward(self, x):
        if self.use_bn:
            x = self.bn(x)
        out = self.features(x)
        return out

---

In [12]:
model = EffNet(num_classes=config['CLASSES'], use_bn=config['USE_BN'],
               use_angular=config['USE_ANGULAR'])
model.to(config['DEVICE']);

In [12]:
criterion = nn.CrossEntropyLoss()
# criterion = AMSoftmaxLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['LR'])

In [13]:
metrics = {
    'loss': Loss(criterion),
    'accuracy': Accuracy(),
}

trainer = create_supervised_trainer(model, optimizer, criterion, device=config['DEVICE'])
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=config['DEVICE'])

In [14]:
# Attach to our trainer a function to run a validator at the end of each epoch
@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_val_metrics(engine):
    epoch = engine.state.epoch
    metrics = val_evaluator.run(val_loader).metrics
    print("Validation Results - Epoch: {}  Average Loss: {:.4f} | Accuracy: {:.4f} "
          .format(engine.state.epoch, 
                      metrics['loss'], 
                      metrics['accuracy']))

In [15]:
# In this problem I think it's better not to use the same learning rate during all the training
# let's make it decrease after each epoch
lr_scheduler = ExponentialLR(optimizer, gamma=0.99)

@trainer.on(Events.EPOCH_COMPLETED)
def update_lr_scheduler(engine):
    lr_scheduler.step()
    lr = float(optimizer.param_groups[0]['lr'])
    print("Learning rate: {}".format(lr))

In [16]:
# We definitely need early stopping
# I don't want to tune the number of epochs by hands
handler = EarlyStopping(patience=6, score_function=lambda engine: engine.state.metrics['accuracy'], trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, handler)

In [17]:
# Also, let's save our model's weights after some epochs to be able to use them later
checkpoints = ModelCheckpoint(model_name, 'all_exps', save_interval=1, n_saved=10, create_dir=True, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoints, {model_name+'_lr{}'.format(lr_str): model})

In [18]:
# And we obviously need beautiful tqdm-based progress bars for our training process
pbar = ProgressBar(bar_format='')
pbar.attach(trainer, output_transform=lambda x: {'loss': x})

In [19]:
# Let's log some interesting information about our learning process to Tensorboard
# (Does not work in kaggle kernels, you need to have TensorboadX installed)

import os
if not 'KAGGLE_WORKING_DIR' in os.environ:  #  If we are not on kaggle server
    from ignite.contrib.handlers.tensorboard_logger import *
    tb_logger = TensorboardLogger("board/"+model_name)
    tb_logger.attach(trainer, log_handler=OutputHandler(tag="training", output_transform=lambda loss: {'loss': loss}),
                     event_name=Events.ITERATION_COMPLETED)

    tb_logger.attach(val_evaluator, log_handler=OutputHandler(tag="validation", metric_names=["accuracy", "loss"],
                     another_engine=trainer),event_name=Events.EPOCH_COMPLETED)

    tb_logger.attach(trainer, log_handler=OptimizerParamsHandler(optimizer), event_name=Events.ITERATION_STARTED)

    tb_logger.attach(trainer, log_handler=GradsHistHandler(model), event_name=Events.EPOCH_COMPLETED)
    tb_logger.close()

In [20]:
trainer.run(train_loader, max_epochs=20)

Validation Results - Epoch: 1  Average Loss: 6.6567 | Accuracy: 0.0030 
Learning rate: 0.00099


Validation Results - Epoch: 2  Average Loss: 6.5074 | Accuracy: 0.0118 
Learning rate: 0.0009801


Validation Results - Epoch: 3  Average Loss: 5.8975 | Accuracy: 0.0496 
Learning rate: 0.0009702990000000001


Validation Results - Epoch: 4  Average Loss: 5.5050 | Accuracy: 0.0739 
Learning rate: 0.0009605960099999999


Validation Results - Epoch: 5  Average Loss: 4.9016 | Accuracy: 0.1309 
Learning rate: 0.0009509900498999999


...

Validation Results - Epoch: 8  Average Loss: 4.9698 | Accuracy: 0.1558 
Learning rate: 0.0009227446944279201


Validation Results - Epoch: 9  Average Loss: 4.0195 | Accuracy: 0.2511 
Learning rate: 0.0009135172474836409


Validation Results - Epoch: 10  Average Loss: 3.6459 | Accuracy: 0.2957 
Learning rate: 0.0009043820750088044


Validation Results - Epoch: 11  Average Loss: 3.0174 | Accuracy: 0.3896 
Learning rate: 0.0008953382542587164


KeyboardInterrupt: 

## Prediction for test

In [12]:
n_epochs = 13

model = EffNet(num_classes=config['CLASSES'], use_bn=config['USE_BN'],
               use_angular=config['USE_ANGULAR'])
checkpoint = torch.load('{0}/all_exps_{0}_lr{1}_{2}.pth'.format(config['MODEL_NAME'],
                                                                config['LR_STR'],
                                                                n_epochs))
model.load_state_dict(checkpoint)
model.to(device)
model.eval();

In [13]:
with torch.no_grad():
    predicted = []  # predicted = np.empty(0)
    for (x1, id1), (x2, id2) in tqdm_notebook(zip(test_loader_1, test_loader_2)):
        x1 = x1.to(device)
        output1 = model(x1)
        
        x2 = x2.to(device)
        output2 = model(x2)
        
        result = 0.5*(output1 + output2)
        predicted.append(result.cpu().numpy())

In [14]:
predicted = np.stack(predicted).squeeze()
predicted.shape

(19897, 1108)

In [15]:
submission = pd.read_csv(path_data + '/test.csv')
submission['sirna'] = np.argmax(predicted, axis=1).astype(int)
submission.to_csv('submits/{}_{}epochs_lr{}.csv'.format(model_name, n_epochs, lr_str),
                  index=False, columns=['id_code','sirna'])

In [16]:
submission.head()

,id_code,experiment,plate,well,sirna
0,HEPG2-08_1_B03,HEPG2-08,1,B03,468
1,HEPG2-08_1_B04,HEPG2-08,1,B04,265
2,HEPG2-08_1_B05,HEPG2-08,1,B05,1100
3,HEPG2-08_1_B06,HEPG2-08,1,B06,312
4,HEPG2-08_1_B07,HEPG2-08,1,B07,968
